In [76]:
import numpy as np
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from multiprocessing import Pool as ThreadPool
from datetime import datetime
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.utils import shuffle
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
import keras
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout, Flatten,Activation,LeakyReLU,PReLU,Lambda,Add,LSTM,Bidirectional,GRU,Conv1D,GlobalAveragePooling1D,GlobalMaxPool1D,GlobalMaxPooling1D,TimeDistributed
from keras.layers.embeddings import Embedding
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import load_model
from keras.utils import multi_gpu_model
#进行配置，每个GPU使用60%上限现存
config = tf.ConfigProto()
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
# config.gpu_options.per_process_gpu_memory_fraction = 0.5 # 每个GPU现存上届控制在60%以内
session = tf.Session(config=config)
# 设置session
KTF.set_session(session )



Using TensorFlow backend.


In [3]:
data_dir='/home/uniml/work/hw/dataset/'
temp_save_dir='./temp_data/'
age_train_file_path=data_dir+'age_train.csv'
age_test_file_path=data_dir+'age_test.csv'
app_info_file_path=data_dir+'app_info.csv'
user_app_actived_file_path=data_dir+'user_app_actived.csv'
user_basic_info_file_path=data_dir+'user_basic_info.csv'
user_behavior_info_file_path=data_dir+'user_behavior_info.csv'
user_app_usage_file_path=data_dir+'user_app_usage.csv'

In [4]:
#############用户label数据与测试数据################

In [4]:
age_train=pd.read_csv(age_train_file_path,header=None,names=['uId','age_group'])
age_train.age_group=age_train.age_group.apply(lambda x:x-1)
age_train.age_group=age_train.age_group.astype('category')
age_test=pd.read_csv(age_test_file_path,header=None,names=['id'])

In [5]:
###########用户基本数据#################

In [156]:
user_basic_info=pd.read_csv(user_basic_info_file_path,header=None,names=['uId',
                                                                         'gender',
                                                                         'city',
                                                                         'prodName',#手机型号
                                                                         'ramCapacity',#手机ram容量
                                                                         'ramLeftRation',#ram剩余容量占比
                                                                         'romCapacity',#ROM容量
                                                                         'romLeftRation',#rom剩余容量占比
                                                                         'color',#手机颜色
                                                                         'fontSize',#字体大小
                                                                         'ct',#上网类型
                                                                         'carrier',#移动运营商
                                                                         'os',#手机系统版本
                                                                        ])

In [7]:
user_basic_info['gender']=user_basic_info['gender'].astype(np.int8)
#增加新特征
big_city=list(user_basic_info['city'].value_counts().iloc[:15].index)
user_basic_info['is_in_big_city']=user_basic_info['city'].apply(lambda x:x in big_city).astype(np.int8)
user_basic_info.loc[pd.isna(user_basic_info['fontSize']),'fontSize']=1
user_basic_info['is_bigger']=user_basic_info['fontSize'].apply(lambda x:x>1.0).astype(np.int8)
user_basic_info['is_smaller']=user_basic_info['fontSize'].apply(lambda x:x<1.0).astype(np.int8)
user_basic_info['is_old_system']=user_basic_info['os'].apply(lambda x:x<=6.0).astype(np.int8)
#处理异常值
user_basic_info.loc[user_basic_info['romCapacity']==137,'romCapacity']=128
user_basic_info.loc[user_basic_info['romCapacity']==68,'romCapacity']=64
user_basic_info.loc[user_basic_info['romCapacity']==34,'romCapacity']=32
user_basic_info.loc[user_basic_info['romCapacity']==275,'romCapacity']=256
user_basic_info.loc[user_basic_info['romCapacity']==550,'romCapacity']=512
user_basic_info.loc[user_basic_info['romCapacity']==17,'romCapacity']=16
user_basic_info.loc[user_basic_info['romCapacity']==3,'romCapacity']=4
user_basic_info.loc[user_basic_info['os']==12.1,'os']=8
user_basic_info.loc[user_basic_info['romLeftRation']>1,'romLeftRation']=user_basic_info\
                                                                    .loc[user_basic_info['romLeftRation']>1,:]\
                                                            .apply(lambda x:x.romLeftRation/x.romCapacity,axis=1)
#去除无用特征
_=user_basic_info.pop('ct')

#更改类型
for column_name in ['gender','city','prodName','color','carrier']:
    user_basic_info[column_name]=user_basic_info[column_name].astype('category')

#处理空缺值及新增部分特征    
user_phone_info_map={}
for name,group in user_basic_info.groupby('prodName'):
    user_phone_info_map[name]={}
    user_phone_info_map[name]['ramCapacity']=group['ramCapacity'].mean()
    user_phone_info_map[name]['romCapacity']=group['romCapacity'].mean()
    user_phone_info_map[name]['ramLeftRation']=group['ramLeftRation'].mean()
    user_phone_info_map[name]['romLeftRation']=group['romLeftRation'].mean()
    user_phone_info_map[name]['ramMax']=group['ramCapacity'].max()
    user_phone_info_map[name]['ramMin']=group['ramCapacity'].min()
    user_phone_info_map[name]['romMax']=group['romCapacity'].max()
    user_phone_info_map[name]['romMin']=group['romCapacity'].min()
def fill_na(y,column_name):
    return user_phone_info_map[y][column_name]
for column_name in ['ramCapacity','romCapacity','ramLeftRation','romLeftRation']:
    user_basic_info.loc[pd.isna(user_basic_info[column_name]),column_name]=\
            user_basic_info.loc[pd.isna(user_basic_info[column_name]),'prodName'].apply(lambda x:fill_na(x,column_name))
for column_name in ['ramMax','ramMin','romMax','romMin']:
    user_basic_info[column_name]=user_basic_info['prodName'].apply(lambda x:fill_na(x,column_name))
for column_name in ['ramCapacity','romCapacity','ramLeftRation','romLeftRation','ramMax','ramMin','romMax','romMin','city','os']:
    user_basic_info[column_name] = user_basic_info[column_name].fillna(user_basic_info[column_name].mode()[0])
user_basic_info.loc[user_basic_info['ramLeftRation']>1,'ramLeftRation']=user_basic_info.loc[user_basic_info['ramLeftRation']>1,:].apply(lambda x:x['ramLeftRation']/x['ramCapacity'],axis=1)
user_basic_info['ramLeft']=((1-user_basic_info['ramLeftRation'])*user_basic_info['ramCapacity']).astype(np.float32)
user_basic_info['romLeft']=((1-user_basic_info['romLeftRation'])*user_basic_info['romCapacity']).astype(np.float32)

###转为labelencoder
cols = ['city','prodName','color','carrier','gender']
for c in cols:
    lbl = LabelEncoder()   
    lbl.fit(list(user_basic_info[c].values)) 
    user_basic_info[c] = lbl.transform(list(user_basic_info[c].values))


In [8]:
############用户行为数据################

In [158]:
user_behavior_info=pd.read_csv(user_behavior_info_file_path,header=None,names=['uId',
                                                                               'bootTimes',
                                                                               'AFuncTimes',
                                                                               'BFuncTimes',
                                                                               'CFuncTimes',
                                                                               'DFuncTimes',
                                                                               'EFuncTimes',
                                                                               'FFuncTimes',
                                                                               'GFuncTimes',
                                                                              ],dtype={'uId':np.int32,
                                                                                      'bootTimes':np.int32,
                                                                               'AFuncTimes':np.float32,
                                                                               'BFuncTimes':np.float32,
                                                                               'CFuncTimes':np.float32,
                                                                               'DFuncTimes':np.float32,
                                                                               'EFuncTimes':np.float32,
                                                                               'FFuncTimes':np.float32,
                                                                               'GFuncTimes':np.int32,})

In [159]:
user_behavior_info.describe()

,uId,bootTimes,AFuncTimes,BFuncTimes,CFuncTimes,DFuncTimes,EFuncTimes,FFuncTimes,GFuncTimes
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,1.076118e+07,1.919097e+01,5.774751e-01,1.302351e-01,1.557342e+00,2.541633e+00,2.132084e+00,1.520421e-01,8.938465e+03
std,7.444156e+06,2.844506e+01,1.089338e+00,1.295618e+00,7.601761e+00,1.171342e+01,9.934189e+00,1.110676e+00,8.224821e+04
min,1.000006e+06,0.000000e+00,0.000000e+00,-1.182000e+03,-1.825500e+03,-1.000000e+03,-1.000000e+03,0.000000e+00,-1.000000e+00
25%,4.745554e+06,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.051819e+06,1.000000e+01,0.000000e+00,0.000000e+00,3.000000e-01,6.700000e-01,3.300000e-01,0.000000e+00,3.390000e+02
75%,1.540960e+07,2.200000e+01,1.000000e+00,7.000000e-02,1.500000e+00,3.000000e+00,2.600000e+00,0.000000e+00,4.313000e+03
max,4.455089e+07,4.500000e+02,8.000000e+00,1.500000e+02,1.500000e+02,1.000000e+03,1.000000e+03,2.000000e+01,4.232854e+07


In [10]:
user_behavior_info=user_behavior_info.applymap(lambda x:max(0,x))
for column_name in user_behavior_info.columns:
    if column_name=='uId':
        continue
    p99= user_behavior_info[column_name].quantile(0.99)
    user_behavior_info.loc[user_behavior_info[column_name]>p99,column_name]=p99

In [11]:
#####################App分类数据##################

In [12]:
app_info=pd.read_csv(app_info_file_path,header=None,names=['appId','category'],dtype={'appId':str,'category':'category'})
app_category_list=list(app_info.category.unique())

In [13]:
############top安装app###############

In [14]:
# pd_app_use_count=pd.read_csv('./temp_data/app_use_count.csv')
# temp_sorted_app_use_count=pd_app_use_count.sort_values(by='count',ascending=False).iloc[0:100,:]
# temp_sorted_app_use_count=temp_sorted_app_use_count.reset_index()
# def get_top_app_id(data):
#     app_rank_map={}
#     for index,row in data.iterrows():
#         app_rank_map[row['appId']]=index
#     return app_rank_map
# top_500_app=get_top_app_id(temp_sorted_app_use_count)

In [15]:
#######用户激活app统计###########

In [16]:
# final_user_app_actived_read_map={}
# for name in app_category_list:
#     final_user_app_actived_read_map[name]=np.int32
#     final_user_app_actived_read_map[name+'_rate']=np.float32
# for app_name in top_500_app.keys():
#     final_user_app_actived_read_map[app_name]=np.int8
# final_user_app_actived_read_map['uId']=np.int32
# final_user_app_actived_read_map['app_count']=np.int32
# final_user_app_actived=pd.read_csv(temp_save_dir+'final_user_app_actived.csv',dtype=final_user_app_actived_read_map)

In [17]:
######top使用时长app#########

In [18]:
# pd_user_app_usage_appid_count=pd.read_csv(temp_save_dir+'pd_user_app_usage_appid_count.csv')
# top_usage_app=pd_user_app_usage_appid_count.sort_values(by='count',ascending=False).iloc[0:500,:]
# top_usage_app=get_top_app_id(top_usage_app)

In [19]:
####用户使用时长app统计#######

In [20]:
# user_app_usage_statistic_dtype_map={}
# for app_category_name in app_category_list:
#     user_app_usage_statistic_dtype_map[app_category_name+'_times']=np.float32
#     user_app_usage_statistic_dtype_map[app_category_name+'_duration']=np.float32
#     user_app_usage_statistic_dtype_map[app_category_name+'_avg']=np.float32
# user_app_usage_statistic_dtype_map['uId']=np.int32
# user_app_usage_statistic_dtype_map['all_times']=np.int32
# user_app_usage_statistic_dtype_map['all_duration']=np.int32
# user_app_usage_statistic_dtype_map['use_days']=np.int32
# user_app_usage_statistic_dtype_map['app_usage_count']=np.int32
# pd_user_app_usage=pd.read_csv(temp_save_dir+'user_app_usage_statistic_fifth_version_sum_simply_version.csv',dtype=user_app_usage_statistic_dtype_map)

In [21]:
final_user_app_actived=pd.read_hdf(temp_save_dir+'final_user_app_actived.h5',key='data')

In [22]:
for i in final_user_app_actived.columns:
    if i.endswith('_rate'):
        _=final_user_app_actived.pop(i)

In [23]:
pd_user_app_usage=pd.read_hdf(temp_save_dir+'user_app_usage_statistic.h5',key='data')

In [24]:
for i in pd_user_app_usage.columns:
    if i.endswith('_avg'):
        _=pd_user_app_usage.pop(i)

In [25]:
##############app激活list################

In [26]:
actived_app_size = 10010
actived_max_length = 150

user_app_actived=pd.read_csv(user_app_actived_file_path,header=None,names=['uId','appId'])

encoded_apps = [one_hot(app[0], actived_app_size, split='#') for app in user_app_actived[['appId']].values]

padded_apps = pad_sequences(encoded_apps, maxlen=actived_max_length, padding='post')
padded_apps = np.array(padded_apps)

actived_padded_apps_uId = pd.concat([user_app_actived,pd.DataFrame(padded_apps,dtype=int)], axis=1)
_ = actived_padded_apps_uId.pop('appId')

In [27]:
##############app使用list################

In [28]:
usage_app_size = 10010
usage_max_length = 200

app_usage_info=pd.read_hdf(data_dir+'user_app_usage_df.h5',key='data')
app_usage_info.loc[pd.isna(app_usage_info['appIds']),'appIds']=''

encoded_apps = [one_hot(app[0], usage_app_size, split='#') for app in app_usage_info[['appIds']].values]

padded_apps = pad_sequences(encoded_apps, maxlen=usage_max_length, padding='post')
padded_apps = np.array(padded_apps)

padded_apps_uId = pd.concat([app_usage_info,pd.DataFrame(padded_apps,dtype=int)], axis=1)
_ = padded_apps_uId.pop('appIds')

In [29]:
#########embedding###########

In [30]:
embedding_dtype_map={'uId':np.int32}
for i in range(32):
    embedding_dtype_map[str(i)+'_sum']=np.float32
    embedding_dtype_map[str(i)+'_avg']=np.float32

In [31]:
app_train_embedding_sum=pd.read_hdf(temp_save_dir+'app_actived_embedding_by_w2v_avg_windows_5_size_128_iter_20_1.h5',key='data')
app_usage_embedding_sum=pd.read_hdf(temp_save_dir+'app_usage_embedding_by_w2v_sum_windows_5_size_128_iter_20_1.h5',key='data')


In [32]:
# for emb in [app_train_embedding_sum,
# #             app_train_embedding_avg,
#             app_usage_embedding_sum,
# #             app_usage_embedding_avg
#            ]:#,\
#            #cbow_app_train_embedding_sum,cbow_app_train_embedding_avg,cbow_app_usage_embedding_sum,cbow_app_usage_embedding_avg]:
#     need_norm=list(emb.columns)
#     need_norm.remove('uId')
#     norm=np.linalg.norm(emb[need_norm],axis=1,keepdims=True)
#     print('start divid ')
#     emb[need_norm]=np.array(emb[need_norm])/norm
#     emb['uId']=emb['uId'].astype(np.int32)
#     #emb['norm']=pd.DataFrame(norm)
#     print('finish')
    

In [33]:
###########lstm输入数据

In [34]:
# train_lstm_input = pd.read_hdf(data_dir+'./train_lstm_input.h5',key='data')
# test_lstm_input = pd.read_hdf(data_dir+'./test_lstm_input.h5',key='data')

In [35]:
#########yc的26维统计特征

In [36]:
train_yc_static=pd.read_hdf(data_dir+'train_app_usage_stat_df.h5',key='data')
test_yc_static=pd.read_hdf(data_dir+'test_app_usage_stat_df.h5',key='data')

In [37]:
#########usage数据以及转化率

In [38]:
train_app_trans_rate_with_usage=pd.read_hdf(data_dir+'train_app_trans_rate_with_usage.h5',key='data')
test_app_trans_rate_with_usage=pd.read_hdf(data_dir+'test_app_trans_rate_with_usage.h5',key='data')

In [39]:
##########构建训练集#######

In [40]:
numeric_input=pd.merge(age_train[['uId']],user_basic_info,on='uId',how='left')
numeric_input=pd.merge(numeric_input,user_behavior_info,on='uId',how='left')
numeric_input=pd.merge(numeric_input,final_user_app_actived,on='uId',how='left')
numeric_input=pd.merge(numeric_input,pd_user_app_usage,on='uId',how='left')
numeric_input.fillna(0,inplace=True)
_=numeric_input.pop('uId')

In [41]:
train_gender_input=numeric_input.pop('gender')
train_city_input=numeric_input.pop('city')
train_prodName_input=numeric_input.pop('prodName')
train_color_input=numeric_input.pop('color')
_=numeric_input.pop('carrier')

In [42]:
# top_feature_select=SelectKBest(chi2,k=150)
# top_feature_select.fit(numeric_input,age_train['age_group'])
# numeric_input=top_feature_select.transform(numeric_input)

In [43]:
scaler = StandardScaler().fit(numeric_input)
numeric_input = scaler.transform(numeric_input)  

In [44]:
numeric_input.shape

(4000000, 125)

In [45]:
# train_app_actived_input=app_actived_train_inputs
train_app_actived_input=pd.merge(age_train[['uId']],actived_padded_apps_uId,on='uId',how='left')
train_app_actived_input.fillna(0,inplace=True)
_=train_app_actived_input.pop('uId')

In [46]:
train_app_usage_input=pd.merge(age_train[['uId']],padded_apps_uId,on='uId',how='left')
train_app_usage_input.fillna(0,inplace=True)
_=train_app_usage_input.pop('uId')

In [47]:
# train_lstm_input=train_lstm_input.values.reshape((-1,7,200))

In [48]:
train_emb_list=[]
emb_scaler=[]

In [49]:
for emb in [app_train_embedding_sum,
#             app_train_embedding_avg,
            app_usage_embedding_sum,
#             app_usage_embedding_avg
           ]:#,\
    train_emb=pd.merge(age_train[['uId']],emb,on='uId',how='left')
    train_emb.fillna(0,inplace=True)
    _=train_emb.pop('uId')
    train_emb_scaler=StandardScaler().fit(train_emb)
    train_emb=train_emb_scaler.transform(train_emb)
    train_emb_list.append(train_emb)
    emb_scaler.append(train_emb_scaler)

In [50]:
yc_scalar=StandardScaler().fit(train_yc_static)
train_yc_static=yc_scalar.transform(train_yc_static)  

In [51]:
#########神经网络

In [167]:
def build_res_net():
    inputs = []
    city_embedding_size = 20
    prodName_embedding_size = 15
    color_embedding_size = 10
    gender_color_embedding_size = 15
    app_actived_size = 30
    app_usage_size = 30
    
    city_input = Input(shape=(1,))
    city_embedded = Embedding(343, city_embedding_size, input_length=1)(city_input)
    inputs.append(city_input)
    
    prodName_input = Input(shape=(1,))
    prodName_embedded = Embedding(114, prodName_embedding_size, input_length=1)(prodName_input)
    inputs.append(prodName_input)
    
    color_input = Input(shape=(1,))
    color_embedded = Embedding(117, color_embedding_size, input_length=1)(color_input)
    inputs.append(color_input)
    
    gender_color_input = Input(shape=(1,))
    gender_color_embedded = Embedding(266, gender_color_embedding_size, input_length=1)(gender_color_input)
    inputs.append(gender_color_input)
    
    input_numeric = Input(shape=(125,))
    inputs.append(input_numeric)
    
    
    
    app_actived_input = Input(shape=(150,))
    app_actived_embedded = Embedding(10010,app_actived_size,input_length=150)(app_actived_input)
    app_actived_embedded_layer = Lambda(lambda x:K.sum(x,axis=1))(app_actived_embedded)
    inputs.append(app_actived_input)


  
    
    
    app_usage_input = Input(shape=(200,))
    app_usage_embedded = Embedding(10010,app_usage_size,input_length=200)(app_usage_input)
    app_usage_embedded_layer = Lambda(lambda x:K.sum(x,axis=1))(app_usage_embedded)
    inputs.append(app_usage_input)
    
    

#     gru_input = Input(shape=(7,200))
#     gru = Bidirectional(GRU(64,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(gru_input)
#     gru = Bidirectional(GRU(32,dropout=0.2,recurrent_dropout=0.2))(gru)
#     inputs.append(gru_input)




    yc_static_input=Input(shape=(26,))
    inputs.append(yc_static_input)
    
    usgae_trans_rate_input=Input(shape=(66,))
    inputs.append(usgae_trans_rate_input)
    
    actived_emb_sum_input=Input(shape=(128,))
    inputs.append(actived_emb_sum_input)
    

    
    usgae_emb_sum_input=Input(shape=(128,))
    inputs.append(usgae_emb_sum_input)

        
    
    con = Concatenate()([city_embedded, prodName_embedded, color_embedded,gender_color_embedded])
    flatten = Flatten()(con)
    x = Concatenate()([flatten, 
                       input_numeric, 
                       app_actived_embedded_layer, app_usage_embedded_layer,
                       actived_emb_sum_input,
                       usgae_emb_sum_input,
#                         gru,
                       yc_static_input,
                       usgae_trans_rate_input,
                      ])
    
    x = Dense(1024)(x)
    y = BatchNormalization()(x)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Dense(1024)(y)
    y = BatchNormalization()(y)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    residual = x
    y = Add()([y, residual])
    
    x = Dense(1024)(y)
    residual = y
    y = BatchNormalization()(x)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Dense(1024)(y)
    y = BatchNormalization()(y)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Add()([y, residual])
    
    x = Dense(1024)(y)
    residual = y
    y = BatchNormalization()(x)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Dense(1024)(y)
    y = BatchNormalization()(y)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Add()([y, residual])
    
    x = Dense(1024)(y)
    residual = y
    y = BatchNormalization()(x)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Dense(1024)(y)
    y = BatchNormalization()(y)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Add()([y, residual])
    
    x = Dense(128)(y)
    y = BatchNormalization()(x)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    y = Dense(128)(y)
    y = BatchNormalization()(y)
    y = PReLU()(y)
    y = Dropout(0.2)(y)
    residual = x
    y = Add()([y, residual])
    
    output = Dense(1)(y)
#     output = Activation('softmax',name='cate_out')(output)
    model = Model(inputs, output)
    model.compile(loss='mse', optimizer=optimizers.Adam(lr=0.001),metrics=['accuracy'])
    return model

In [163]:
model_name='my_model_18.h5'

In [54]:
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=20
    ),
    ModelCheckpoint(
        filepath=model_name, 
        monitor='val_loss', 
        save_best_only=True
    )
]

In [55]:
####训练集

In [56]:
X = [train_city_input,train_prodName_input,train_color_input,train_gender_input,numeric_input,train_app_actived_input.values,train_app_usage_input.values,train_yc_static,train_app_trans_rate_with_usage.values]+train_emb_list

In [57]:
Y = np_utils.to_categorical(age_train['age_group'].values, 6)

In [160]:
num_Y=np.argmax(Y,axis=1)
pd_Y=pd.DataFrame(num_Y)
index_set=[]
from sklearn.utils import shuffle
index_set.extend(list(shuffle(pd_Y[pd_Y[0]==0]).index)[:100000])
index_set.extend(list(shuffle(pd_Y[pd_Y[0]==1]).index)[:500000])
index_set.extend(list(shuffle(pd_Y[pd_Y[0]==2]).index)[:800000])
index_set.extend(list(shuffle(pd_Y[pd_Y[0]==3]).index)[:700000])
index_set.extend(list(shuffle(pd_Y[pd_Y[0]==4]).index)[:400000])
index_set.extend(list(shuffle(pd_Y[pd_Y[0]==5]).index)[:200000])
print(len(index_set))
index_set=shuffle(index_set)

2700000


In [142]:
index_set

[703011,
 851386,
 837124,
 723267,
 525114,
 597296,
 578696,
 599023,
 557917,
 759208,
 641856,
 805389,
 683302,
 932792,
 610059,
 751625,
 688137,
 574990,
 653936,
 923904,
 817685,
 826377,
 582096,
 636636,
 891483,
 833609,
 647542,
 830804,
 841272,
 695597,
 908626,
 580537,
 930404,
 987003,
 739394,
 983626,
 666335,
 838239,
 717713,
 561525,
 863227,
 631547,
 531004,
 682082,
 850714,
 649942,
 696366,
 736034,
 752786,
 887207,
 659139,
 812753,
 961907,
 517489,
 978819,
 602407,
 944790,
 539758,
 905575,
 792534,
 897713,
 768205,
 746414,
 536374,
 917046,
 615084,
 813655,
 669433,
 702907,
 580264,
 761356,
 869497,
 788423,
 763784,
 754936,
 742289,
 747336,
 635036,
 679436,
 800664,
 713620,
 808355,
 743631,
 823605,
 641001,
 979665,
 545881,
 645032,
 951278,
 701052,
 772514,
 648406,
 836277,
 575467,
 794432,
 679842,
 719560,
 986541,
 509991,
 653561,
 895258,
 749669,
 679012,
 967420,
 951139,
 814172,
 940100,
 501678,
 805689,
 613294,
 969467,
 

In [58]:
###测试集

In [59]:
test_numeric_input=pd.merge(age_test,user_basic_info,left_on=['id'],right_on=['uId'],how='left')
test_numeric_input=pd.merge(test_numeric_input,user_behavior_info,on='uId',how='left')
test_numeric_input=pd.merge(test_numeric_input,final_user_app_actived,on='uId',how='left')
test_numeric_input=pd.merge(test_numeric_input,pd_user_app_usage,on='uId',how='left')
test_numeric_input.fillna(0,inplace=True)
_=test_numeric_input.pop('uId')
_=test_numeric_input.pop('id')

In [60]:
test_gender_input=test_numeric_input.pop('gender')
test_city_input=test_numeric_input.pop('city')
test_prodName_input=test_numeric_input.pop('prodName')
test_color_input=test_numeric_input.pop('color')
_=test_numeric_input.pop('carrier')

In [61]:
# test_numeric_input=top_feature_select.transform(test_numeric_input)

In [62]:
test_numeric_input = scaler.transform(test_numeric_input)

In [63]:
# test_app_actived_input=app_actived_test_inputs
test_app_actived_input=pd.merge(age_test,actived_padded_apps_uId,left_on=['id'],right_on=['uId'],how='left')
test_app_actived_input.fillna(0,inplace=True)
_=test_app_actived_input.pop('uId')
_=test_app_actived_input.pop('id')

In [64]:
test_app_usage_input=pd.merge(age_test,padded_apps_uId,left_on=['id'],right_on=['uId'],how='left')
test_app_usage_input.fillna(0,inplace=True)
_=test_app_usage_input.pop('uId')
_=test_app_usage_input.pop('id')

In [65]:
# test_lstm_input =test_lstm_input.values.reshape((-1,7,200))

In [66]:
test_emb_list=[]

In [67]:
for index,emb in enumerate([
    app_train_embedding_sum,
#     app_train_embedding_avg,
    app_usage_embedding_sum,
#     app_usage_embedding_avg
]):
    test_emb=pd.merge(age_test,emb,left_on=['id'],right_on=['uId'],how='left')
    test_emb.fillna(0,inplace=True)
    _=test_emb.pop('uId')
    _=test_emb.pop('id')
    test_emb=emb_scaler[index].transform(test_emb)
    test_emb_list.append(test_emb)

In [68]:
test_yc_static=yc_scalar.transform(test_yc_static)  

In [69]:
test_X = [test_city_input,test_prodName_input,test_color_input,test_gender_input,test_numeric_input,test_app_actived_input.values,test_app_usage_input.values,test_yc_static,test_app_trans_rate_with_usage.values]+test_emb_list

In [70]:
len(X)

11

In [71]:

class ParallelModelCheckpoint( keras.callbacks.ModelCheckpoint):
    def __init__(self,model,filepath, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1):
        self.single_model = model
        super(ParallelModelCheckpoint,self).__init__(filepath, monitor, verbose,save_best_only, save_weights_only,mode, period)

    def set_model(self, model):
        super(ParallelModelCheckpoint,self).set_model(self.single_model)

In [89]:
row_indices = np.random.permutation(Y.shape[0])
split_size=5
step=int(Y.shape[0]/float(split_size))
split_index_list=[]
for i in range(split_size):
    start=i*step
    end=(i+1)*step
    print(start,end)
    split_index_list.append(row_indices[start:end])

0 800000
800000 1600000
1600000 2400000
2400000 3200000
3200000 4000000


In [165]:
# row_indices = np.random.permutation(Y.shape[0])
train_x=[x_data[index_set] for x_data in X]
train_y=Y[index_set]
train_y=np.argmax(train_y,axis=1)

In [169]:
model_name='test_single_model_sample'+'.h5'
    
model = build_res_net()
#     callbacks_list = [
#     EarlyStopping(
#         monitor='val_loss',
#         patience=20
#     ),
#     ModelCheckpoint(
#         filepath=model_name, 
#         monitor='val_loss', 
#         save_best_only=True
#     )
# ]
#     model.fit(train_x, train_y, epochs=100,batch_size=4096,validation_data=[validation_x,validation_y],callbacks=callbacks_list
callbacks_list = [
     EarlyStopping(
         monitor='val_loss',
         patience=10
     ),
     ParallelModelCheckpoint(model,model_name,save_best_only=True,monitor='val_loss')
]
 
model_parallel=multi_gpu_model(model,gpus=4)
model_parallel.compile(loss='mse', optimizer=optimizers.Adam(lr=0.001),metrics=['accuracy'])
 
 
 
model_parallel.fit(train_x, train_y, epochs=50,batch_size=8192*4,validation_split=0.1,callbacks=callbacks_list)
 

my_model = load_model(model_name)
y_pred = my_model.predict(test_X)
pred_list.append(y_pred)
 
##备份
pd_y_pred=pd.DataFrame(y_pred)
pd_y_pred.to_csv('pred_nn_test_model_sample'+'.csv',index=False,header=None) 

Train on 2430000 samples, validate on 270000 samples
Epoch 1/50
2430000/2430000 [==============================] - 125s 52us/step - loss: 27.8845 - acc: 0.2200 - val_loss: 2.5877 - val_acc: 0.3732
Epoch 2/50
2430000/2430000 [==============================] - 32s 13us/step - loss: 1.3737 - acc: 0.3976 - val_loss: 0.8168 - val_acc: 0.4860
Epoch 3/50
2430000/2430000 [==============================] - 31s 13us/step - loss: 1.3970 - acc: 0.4123 - val_loss: 1.1195 - val_acc: 0.4436
Epoch 4/50
2430000/2430000 [==============================] - 32s 13us/step - loss: 1.2482 - acc: 0.4274 - val_loss: 0.8126 - val_acc: 0.4845
Epoch 5/50
2430000/2430000 [==============================] - 31s 13us/step - loss: 1.2863 - acc: 0.4365 - val_loss: 0.7812 - val_acc: 0.4998
Epoch 6/50
2430000/2430000 [==============================] - 31s 13us/step - loss: 1.0030 - acc: 0.4573 - val_loss: 1.0662 - val_acc: 0.4681
Epoch 7/50
2430000/2430000 [==============================] - 30s 12us/step - loss: 0.9541 - 

In [88]:
prefix='try_4_'

In [90]:
pred_list=[]
test_x=test_X

for i in range(split_size):
    ####切分数据集
    validation_index=list(split_index_list[i])
    train_index=[]
    for j in range(split_size):
        if j==i:
            continue
        train_index+=list(split_index_list[j])
    train_x=[x_data[train_index] for x_data in X]
    train_y=Y[train_index]  
    validation_x=[x_data[validation_index] for x_data in X]
    validation_y=Y[validation_index]
    print('finish split data')
    
    model_name='my_model_split_'+prefix+str(i)+'.h5'
    
    model = build_res_net()
#     callbacks_list = [
#     EarlyStopping(
#         monitor='val_loss',
#         patience=20
#     ),
#     ModelCheckpoint(
#         filepath=model_name, 
#         monitor='val_loss', 
#         save_best_only=True
#     )
# ]
#     model.fit(train_x, train_y, epochs=100,batch_size=4096,validation_data=[validation_x,validation_y],callbacks=callbacks_list)

    callbacks_list = [
        EarlyStopping(
            monitor='val_loss',
            patience=10
        ),
        ParallelModelCheckpoint(model,model_name,save_best_only=True,monitor='val_loss')

    ]
    
    model_parallel=multi_gpu_model(model,gpus=4)
    model_parallel.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    
    
    model_parallel.fit(train_x, train_y, epochs=10,batch_size=8192*4,validation_data=[validation_x,validation_y],callbacks=callbacks_list)
    
    
    my_model = load_model(model_name)
    y_pred = my_model.predict(test_x)
    pred_list.append(y_pred)
    
    ##备份
    pd_y_pred=pd.DataFrame(y_pred)
    pd_y_pred.to_csv('pred_nn_'+prefix+str(i)+'.csv',index=False,header=None)

    print('finish:',str(i))

finish split data
Train on 3200000 samples, validate on 800000 samples
Epoch 1/10
3200000/3200000 [==============================] - 94s 29us/step - loss: 1.6299 - acc: 0.4833 - val_loss: 1.1359 - val_acc: 0.5718
Epoch 2/10
3200000/3200000 [==============================] - 50s 15us/step - loss: 1.2204 - acc: 0.5450 - val_loss: 1.0674 - val_acc: 0.5880
Epoch 3/10
1507328/3200000 [=============>................] - ETA: 21s - loss: 1.1443 - acc: 0.5640

KeyboardInterrupt: 

In [80]:
result=pred_list[0]+pred_list[1]+pred_list[2]+pred_list[3]+pred_list[4]
result=np.argmax(result,axis=1)+1
result=pd.DataFrame(result)
submission=pd.concat([age_test,result],axis=1)
submission.rename(columns={0:'label'},inplace=True)
submission.to_csv('submission_nn'+prefix+'all.csv',index=False)
# print(accuracy_score(np.argmax(test_y,axis=1)+1,result))
# print(accuracy_score(np.argmax(test_y,axis=1)+1,result))

In [81]:
full_pred=my_model.predict(X)
result=np.argmax(full_pred,axis=1)+1

In [84]:
num_Y=np.argmax(Y,axis=1)+1

In [87]:
sum(result==num_Y)/len(num_Y)

0.642301

In [ ]:
# print(submission.label.value_counts()/len(submission))

In [76]:
# save_file_list=[]
# for i in range(split_size):
#     save_file_list.append(np.array(pd.read_csv('pred_nn_'+prefix+str(i)+'.csv',header=None)))

In [77]:
# result_1=save_file_list[0]+save_file_list[1]+save_file_list[2]+save_file_list[3]+save_file_list[4]
# result_1=np.argmax(result_1,axis=1)+1
# result=pd.DataFrame(result)
# submission=pd.concat([age_test,result],axis=1)
# submission.rename(columns={0:'label'},inplace=True)
# submission.to_csv('pred_nn'+prefix+'all.csv',index=False)
# # print(accuracy_score(np.argmax(test_y,axis=1)+1,result))

In [92]:
model = build_res_net()
# model_parallel=multi_gpu_model(model,gpus=4)
# model_parallel.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])
# class ParallelModelCheckpoint( keras.callbacks.ModelCheckpoint):
#     def __init__(self,model,filepath, monitor='val_loss', verbose=0,
#                  save_best_only=False, save_weights_only=False,
#                  mode='auto', period=1):
#         self.single_model = model
#         super(ParallelModelCheckpoint,self).__init__(filepath, monitor, verbose,save_best_only, save_weights_only,mode, period)

#     def set_model(self, model):
#         super(ParallelModelCheckpoint,self).set_model(self.single_model)
# callbacks_list = [
#     EarlyStopping(
#         monitor='val_loss',
#         patience=20
#     ),
#     ParallelModelCheckpoint(model,model_name,save_best_only=True,monitor='val_loss')
    
# ]

In [93]:
model.fit(X, Y, epochs=100,batch_size=8192*4,validation_split=0.2,callbacks=callbacks_list)

Train on 3200000 samples, validate on 800000 samples
Epoch 1/100
3200000/3200000 [==============================] - 108s 34us/step - loss: 1.4321 - acc: 0.5760 - val_loss: 15.5078 - val_acc: 0.0000e+00
Epoch 2/100
  98304/3200000 [..............................] - ETA: 1:35 - loss: 1.0561 - acc: 0.6240

KeyboardInterrupt: 

In [ ]:
print('da')

In [ ]:
# my_model = load_model(model_name)

In [ ]:
# y_pred = my_model.predict(test_X)

In [ ]:
# pd.DataFrame(y_pred).to_csv('pred_nn_single_model.csv',index=False,header=None)

In [ ]:
print('da')

In [150]:
result=pd_y_pred
result=np.argmax(result.values,axis=1)+1
result=pd.DataFrame(result)
submission=pd.concat([age_test,result],axis=1)
submission.rename(columns={0:'label'},inplace=True)
submission.to_csv('submission_single_model_pred_0_adad.csv',index=False)
print(submission.label.value_counts()/len(submission))

4    0.343963
3    0.330621
5    0.146233
2    0.108920
1    0.050924
6    0.019339
Name: label, dtype: float64


In [154]:
submission_test=pd.read_csv('submission_nntry_3_all.csv')

In [155]:
submission_test.label.value_counts()/len(submission_test)

4    0.293558
3    0.276373
5    0.185773
1    0.107845
2    0.103970
6    0.032481
Name: label, dtype: float64